# Kaggle Titanic 생존자 예측하기

캐글의 튜토리얼 중 하나인 타이타닉호 생존자 예측하기입니다.

https://www.kaggle.com/c/titanic/

## Import Graphlab

graphlab 라이브러리를 import합시다.

In [ ]:
import graphlab

## 데이터 화일 압축 풀기

먼저 kaggle_titanic.zip 화일을 data 폴더 아래에 업로드합니다. 그런 다음 아래 명령을 실행해서 압축을 풉니다.

In [ ]:
#!unzip -o ../data/kaggle_titanic.zip -d ../data

압축이 잘 풀렸는지 꼭 확인합시다.

In [ ]:
#ls ../data/kaggle_titanic/

## 데이터 가져오기


먼저 train.csv 화일을 가져 옵시다. 별도의 검증 데이터가 존재하지 않기 때문에 학습 데이터를 가져와서 train_data와 validation_data로 나눠서 사용하고자 합니다. 따라서 전체 학습 데이터는 **full_train_data**라는 변수에 저장합니다.

    graphlab.SFrame(csv화일이름)

In [ ]:
full_train_data = graphlab.SFrame('../data/kaggle_titanic/train.csv')

head메소드로 데이터를 확인합시다.

In [ ]:
full_train_data.head()

test.csv에 들어 있는 테스트 데이터를 **test_data**에 저장합시다.

In [ ]:
test_data = graphlab.SFrame('../data/kaggle_titanic/test.csv')

마찬가지로 head 메소드로 확인합시다.

In [ ]:
test_data.head()

그래프 시각화를 위해서 아래 명령을 실행합시다.

In [ ]:
graphlab.canvas.set_target('ipynb')

## 데이터 정의

원본은 다음 링크에서 확인하실 수 있습니다.
https://www.kaggle.com/c/titanic/data

<tbody>
<tr><th><b>Variable</b></th><th><b>Definition</b></th><th><b>Key</b></th></tr>
<tr>
<td>survival</td>
<td>Survival</td>
<td>0 = No, 1 = Yes</td>
</tr>
<tr>
<td>pclass</td>
<td>Ticket class</td>
<td>1 = 1st, 2 = 2nd, 3 = 3rd</td>
</tr>
<tr>
<td>sex</td>
<td>Sex</td>
<td></td>
</tr>
<tr>
<td>Age</td>
<td>Age in years</td>
<td></td>
</tr>
<tr>
<td>sibsp</td>
<td># of siblings / spouses aboard the Titanic</td>
<td></td>
</tr>
<tr>
<td>parch</td>
<td># of parents / children aboard the Titanic</td>
<td></td>
</tr>
<tr>
<td>ticket</td>
<td>Ticket number</td>
<td></td>
</tr>
<tr>
<td>fare</td>
<td>Passenger fare</td>
<td></td>
</tr>
<tr>
<td>cabin</td>
<td>Cabin number</td>
<td></td>
</tr>
<tr>
<td>embarked</td>
<td>Port of Embarkation</td>
<td>C = Cherbourg, Q = Queenstown, S = Southampton</td>
</tr>
</tbody>


### 변수 노트

**pclass**: A proxy for socio-economic status (SES)<br>
1st = Upper<br>
2nd = Middle<br>
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...<br>
Sibling = brother, sister, stepbrother, stepsister<br>
Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...<br>
Parent = mother, father<br>
Child = daughter, son, stepdaughter, stepson<br>
Some children travelled only with a nanny, therefore parch=0 for them.<br>

## 데이터 살펴보기


show 메소드로 학습 데이터를 확인합시다.

In [ ]:
full_train_data.show()

shape 메소드로 학습데이터의 전체 길이와 컬럼 개수를 확인합시다.

In [ ]:
full_train_data.shape

총 891명, PassengerId와 Survived 제외하면 총 10개의 컬럼 존재

Pclass 	Name 	Sex 	Age 	SibSp 	Parch 	Ticket 	Fare 	Cabin 	Embarked

- Cabin은 비어 있는 데이터가 많음
- Ticket 값은 알 수 없는 값이 많음

## 성별만으로 예측하기


### 성별 데이터 살펴보기

#### 데이터 전처리
- 비어 있는 값 (없는 값)이 있는지 확인하기. 있다면 값 채우기
- 생존율과의 상관관계 확인하기


먼저 성별의 분포를 알아 봅시다.

In [ ]:
full_train_data['Sex'].show()
test_data['Sex'].show()

다행히 비어 있는 값이 없군요.

생존율의 분포도 확인해 봅시다.

성별과 생존율의 상관 관계를 확인해 봅시다.

In [ ]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="Sex")

In [ ]:
full_train_data[full_train_data['Survived'] == 1].show(view="Bar Chart", x="Sex")

데이터에는 문제가 없어 보이고 생존율과의 상관관계도 어느 정도 있어 보입니다.

먼저 전체 학습 데이터를 학습 데이터와 검증 데이터로 분리합시다. **train_data, validation_data**라는 변수에 저장합시다.

        SFrame.random_split(비율, seed=시드값)
        
시드값은 0을 사용합시다.

In [ ]:
train_data, validation_data = full_train_data.random_split(.8, seed=0)

### 성별 데이터만으로 모델 학습시키기

이제 아래 classifier를 이용해서 모델을 학습시키고 **sex_only_model**에 저장합시다.

    graphlab.logistic_classifier.create(학습데이터, target=타겟컬럼이름, features=피쳐컬럼이름의리스트, validation_set=검증데이터)

In [ ]:
sex_only_model = graphlab.logistic_classifier.create(train_data, target='Survived', features=['Sex'], validation_set =validation_data)

### 검증데이터를 이용해서 모델 검증하기

show 메소드로 모델의 결과를 살펴 봅시다.
    
    model.show()

In [ ]:
sex_only_model.show()

검증 데이터를 이용해서 검증합시다.

    model.evaluate(검증데이터)

In [ ]:
sex_only_model.evaluate(validation_data)

accuracy가 얼마 나왔나요?

위의 코드를 수정해서 accuracy만 출력해 보세요.

In [ ]:
sex_only_model.evaluate(validation_data)['accuracy']

#### (고급) 직접 accuracy 구해보기

먼저 학습된 모델을 이용해서 검증 데이터에 대한 예측값을 구합시다. 생존 확률이 0.5 이상인 경우 생존으로 판별하기로 합니다.

참고: output_type을 지정하지 않으면 0과 1로 판별되어 나옵니다. 아래 코드를 이용하면 생존확률의 기준 (0.5)을 높이거나 낮출 수 있습니다. (예. 0.7)

    model.predict(검증데이터, output_type='probability')

In [ ]:
sex_only_model.predict(validation_data, output_type='probability')

In [ ]:
sex_only_model.predict(validation_data, output_type='probability') >= 0.5

이 결과값을 **predictions**에 저장합시다.

In [ ]:
predictions = sex_only_model.predict(validation_data) >= 0.5

이 예측값을 검증데이터의 'Survived' 컬럼과 비교합시다. 즉 모델이 예측한 값이 검증데이터의 관측값과 얼마나 일치하는지 개수를 세는 겁니다. 

참고로 동일한 값을 확인하는 연산자 (operator)는 **==**입니다.

In [ ]:
predictions == validation_data['Survived']

이 결과값을 **corrects**에 저장합시다.

In [ ]:
corrects = predictions == validation_data['Survived']

정확도는 아래와 같은 식으로 계산할 수 있습니다. 

    accuracy = 맞은 개수 / 전체 개수
    
여기서 맞은 개수는 어떻게 구할 수 있을까요?

In [ ]:
sum(corrects)

138개가 나와야 합니다.

이제 정확도(accuracy)를 구해 봅시다.

In [ ]:
print(sum(corrects)/float(len(validation_data)))

0.734042553191가 나와야 합니다.

다음 방법으로 정확도를 간단히 구할 수 있습니다.

In [ ]:
sex_only_model.evaluate(validation_data)['accuracy']

### Baseline과 비교하기

이제 baseline을 구해서 우리가 만든 모델의 정확도가 의미 있는 것인지 확인해 봅시다.

random_guessing인 경우 얼마일까요?

majority class를 구해 봅시다.

    SFrame.show(view=뷰유형)
    
뷰 유형으로는 'Categorical'을 사용하세요.

어떤 데이터의 어떤 컬럼을 이용해서 majority class를 구해야 할까요?

In [ ]:
full_train_data['Survived'].show(view='Categorical')
full_train_data['Sex'].show(view='Categorical')

우리가 만든 모델의 예측값이 baseline인 남자의 생존율 60.106%보다 월등히 높다는 것을 알 수 있습니다.

## 더 많은 데이터를 이용한 모델 만들기

이제 추가적인 데이터를 이용해서 모델을 만들어 봅시다.


### 데이터 전처리를 위한 변수 만들기

데이터 처리를 하기 전에 데이터를 새로운 변수에 저장합시다. 이제 수정한 결과값은 아래 두 변수에 저장할 겁니다. 

In [ ]:
new_full_train_data = full_train_data
new_test_data = test_data

### Pclass 살펴 보기

먼저 Pclass에 대해서 살펴 봅시다.

In [ ]:
new_full_train_data['Pclass']

In [ ]:
new_test_data['Pclass']

show 메소드를 이용해서 데이터의 분포를 확인합시다.

    SArray.show(view=뷰유형)

In [ ]:
new_full_train_data.show(view='Categorial')

In [ ]:
new_test_data.show(view='Categorial')

값이 없는 데이터가 있는지 확인합시다.

힌트: None을 이용하면 됩니다.

항상 full_train_data와 test_data 모두를 확인해야 합니다. 왜일까요?

In [ ]:
new_full_train_data.show(view='Categorial')

다행히 없군요.

생존율과의 상관 관계를 확인합시다.

In [ ]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="Sex")

In [ ]:
full_train_data[full_train_data['Survived'] == 1].show(view="Bar Chart", x="Pclass")

### Fare 살펴보기

show 메소드로 살펴 봅시다.

In [ ]:
full_train_data['Fare'].show()

#### None 찾아서 값 채우기

값이 없는 데이터가 있는지 확인합시다.

In [ ]:
full_train_data[full_train_data['Age']==None]

test_data에 한명의 데이터가 없다는 것을 알 수 있습니다.

### 비어 있는 값 채우기

비어 있는 값을 어떻게 채우면 좋을까요? 여기에서는 test_data의 평균값을 사용합시다.

먼저 'Fare'의 평균값을 구하세요.

    SArray.mean()

In [ ]:
full_train_data['Age'].mean()

35.627188489208635가 나와야 합니다.

In [ ]:
help(graphlab.SFrame.fillna)

fillna 메소드를 이용해서 None으로 비어 있는 값을 지정할 수 있습니다.

    SFrame.fillna(column=컬럼이름, value=지정하고자하는값)
    
**주의: 반드시 다시 SFrame에 저장해야 합니다.** 여기서는 **new_test_data**라는 새로운 변수명을 사용합시다.

PassengerId가 1044인 승객을 다시 출력해 봅시다.

Fare에 35.627...로 값이 잘 지정된 것을 확인할 수 있습니다.

생존율과의 상관관계를 확인합시다. 

In [ ]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="Fare")

In [ ]:
full_train_data[full_train_data['Survived'] == 1].show(view="Bar Chart", x="Fare")

상관 관계가 있어 보이나요?

### Embarked 살펴보기


show 메소드로 살펴봅시다.

테스트 데이터도 마찬가지로 봅시다.

#### None 찾아서 값 채우기

값이 없는 데이터가 있는지 확인합시다.

full_train_data에 2명의 데이터가 Embarked 값이 없다는 것을 확인할 수 있습니다.
가장 많은 값인 S로 채워봅시다.

먼저 공백을 S로 변환하고 다른 값은 그대로 두는 함수를 만듭니다. 이름은 **fillSforEmbarked**로 합시다.

apply 메소드를 이용해서 Embarked 컬럼에 적용합시다. 이 결과값을 new_full_train_data의 'Embarekd' 컬럼에 저장합시다.

PassengerId 62번 승객의 데이터를 찾아서 Embarked가 잘 지정되었는지 확인합시다.

생존율과의 관계를 확인합시다.

In [ ]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="Embarked")

In [ ]:
full_train_data[full_train_data['Survived'] == 1].show(view="Bar Chart", x="Embarked")

## 4개의 피쳐를 이용해서 모델 학습시키기


위에서 살펴본 4가지 데이터를 이용해서 모델을 학습해 봅시다. 먼저 4가지 데이터 컬럼 이름을 이용해서 **my_features**라는 변수를 지정합시다.

    'Sex', 'Pclass', 'Fare', 'Embarked'

In [ ]:
my_features=['Sex', 'Pclass', 'Fare', 'Embarked']

### 수정한 데이터셋을 나누기

**new_full_train_data**를 **new_train_data**와 **new_validation_data**로 나눕시다.

In [ ]:
new_train_data, new_validation_data = new_full_train_data.random_split(.8, seed=0) 

먼저 가장 간단한 모델을 이용해서 학습시킵시다.

    graphlab.logistic_classifier.create(학습데이터, target=타겟컬럼이름, features=피쳐컬럼이름의리스트, validation_set=검증데이터)
    
**my_features_model**에 저장합시다.

In [ ]:
my_features_model = graphlab.logistic_classifier.create(new_train_data, target='Survived', features=my_features, validation_set =new_validation_data)

**my_features_model**의 검증데이터에 대한 정확도를 출력해 봅시다.

In [ ]:
my_features_model.evaluate(new_validation_data)['accuracy']

**sex_only_model**의 검증데이터에 대한 정확도를 출력해 봅시다.

In [ ]:
sex_only_model.evaluate(new_validation_data)['accuracy']

## 망했어요 vs 아니다

어느 값이 더 큰가요? 모델에 피쳐를 추가하면서 정확도가 나아졌나요?

## 다른 모델들도 학습시키기

graphlab에서 제공하는 다른 classifier들을 이용해서 학습시켜 봅시다.

- Random Forest
- Decision Tree
- SVM
- Nearest Neighbor

https://turi.com/products/create/docs/graphlab.toolkits.classifier.html

### Random Forest

In [ ]:
my_features_model_rf = graphlab.random_forest_classifier.create(new_train_data,
                                                        target='Survived',
                                                        features=my_features,
                                                        validation_set=new_validation_data)

In [ ]:
my_features_model_rf.evaluate(validation_data)['accuracy']

### Decision Tree

In [ ]:
my_features_model_dt = graphlab.decision_tree_classifier.create(new_train_data,
                                                        target='Survived',
                                                        features=my_features,
                                                        validation_set=new_validation_data)

In [ ]:
my_features_model_dt.evaluate(validation_data)['accuracy']

### SVM

In [ ]:
my_features_model_svm = graphlab.svm_classifier.create(new_train_data,
                                                        target='Survived',
                                                        features=my_features,
                                                        validation_set=new_validation_data)

In [ ]:
my_features_model_dt.evaluate(validation_data)['accuracy']

### Nearest Neighbor

In [ ]:
my_features_model_nn = graphlab.nearest_neighbor_classifier.create(new_train_data,
                                                                   target='Survived',
                                                                   features=my_features)

In [ ]:
my_features_model_nn.evaluate(validation_data)['accuracy']

## Age 데이터 처리하기

## Submission

In [ ]:
#model_for_submission = A1_model
#model_for_submission = A1_model_rf
#model_for_submission = A1_model_dt
model_for_submission = A1_model_svm

In [ ]:
predictions = model_for_submission.predict(test_data)

In [ ]:
submission = graphlab.SFrame({"PassengerId": test_data['PassengerId'], "Survived": predictions})

In [ ]:
submission.export_csv('../data/kaggle_titanic/submission2_svm.csv')

In [ ]:
A1_data = new_full_train_data

In [ ]:
A1_data = A1_data.fillna('Age', A1_data['Age'].mean() )

In [ ]:
A1_features = ['Sex', 'Pclass', 'Age', 'SibSp', 'SibSp_New']

In [ ]:
def set_sibsp_new(input_sf):
    if input_sf['SibSp'] > 0:
        if input_sf['Sex'] == 'female':
            return 2
        else: 
            return 1
    else:
        return 0
A1_data['SibSp_New'] = A1_data.apply(lambda x:set_sibsp_new(x))

In [ ]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="SibSp")

In [ ]:
A1_train_data, A1_validation_data = A1_data.random_split(.8, seed=0) 

In [ ]:
A1_model = graphlab.logistic_classifier.create(A1_train_data, target='Survived', features=A1_features, validation_set =A1_validation_data )
A1_model.evaluate(A1_validation_data)['accuracy']

In [ ]:
A1_model_rf = graphlab.random_forest_classifier.create(A1_train_data, target='Survived', features=A1_features, validation_set=A1_validation_data)
A1_model_rf.evaluate(A1_validation_data)['accuracy']

In [ ]:
A1_model_dt = graphlab.decision_tree_classifier.create(A1_train_data, target='Survived', features=A1_features, validation_set=A1_validation_data)
A1_model_dt.evaluate(A1_validation_data)['accuracy']

In [ ]:
A1_model_svm = graphlab.svm_classifier.create(A1_train_data, target='Survived', features=A1_features, validation_set=A1_validation_data)
A1_model_svm.evaluate(A1_validation_data)['accuracy']


In [ ]:
#model_for_submission = A1_model
#model_for_submission = A1_model_rf
#model_for_submission = A1_model_dt
model_for_submission = A1_model_svm

In [ ]:
predictions = model_for_submission.predict(test_data)

In [ ]:
submission = graphlab.SFrame({"PassengerId": test_data['PassengerId'], "Survived": predictions})

In [ ]:
submission.export_csv('../data/kaggle_titanic/submission2_svm.csv')